In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import idx2numpy

In [ ]:
train_images_file = './mnist_dataset/train-images.idx3-ubyte'
train_labels_file = './mnist_dataset/train-labels.idx1-ubyte'
test_images_file = './mnist_dataset/t10k-images.idx3-ubyte'
test_labels_file = './mnist_dataset/t10k-labels.idx1-ubyte'

In [ ]:
X_train = idx2numpy.convert_from_file(train_images_file)
Y_train = idx2numpy.convert_from_file(train_labels_file)
X_test = idx2numpy.convert_from_file(test_images_file)
Y_test = idx2numpy.convert_from_file(test_labels_file)

In [ ]:
X_train = X_train.reshape(X_train.shape[0], -1)

In [ ]:
Y_train

In [ ]:
X_test = X_test.reshape(X_test.shape[0], -1)

In [ ]:
Y_test

In [ ]:
X_train = X_train.T
X_train.shape

In [ ]:
X_test = X_test.T
X_test.shape

In [ ]:
def init_params():
    w1 = np.random.rand(10, 784) - 0.5
    b1 = np.random.rand(10, 1) - 0.5
    w2 = np.random.rand(10, 10) - 0.5
    b2 = np.random.rand(10, 1) - 0.5
    return w1, b1, w2, b2

In [ ]:
def ReLU(Z):
    return np.maximum(0, Z)

def softmax(Z):
    return np.exp(Z) / np.sum(np.exp(Z))

def forward_prop(w1, b1, w2, b2, X):
    Z1 = w1.dot(X) + b1
    A1 = ReLU(Z1)
    Z2 = w2.dot(A1) + b2
    A2 = softmax(Z2)
    return Z1, A1, Z2, A2

In [ ]:
def one_hot(Y):
    one_hot_Y = np.zeros((Y.size, Y.max() + 1))
    one_hot_Y[np.arange(Y.size), Y] = 1
    one_hot_Y = one_hot_Y.T
    return one_hot_Y

def deriv_ReLU(Z):
    return Z > 0

def backward_prop(w2, Z1, A1, Z2, A2, X, Y):
    m = Y.size
    one_hot_Y = one_hot(Y)
    dZ2 = A2 - one_hot_Y
    dw2 = 1 / m * dZ2.dot(A1.T)
    db2 = 1 / m * np.sum(dZ2)
    dZ1 = 1 / m * w2.T.dot(dZ2) * deriv_ReLU(Z1)
    dw1 = 1 / m * dZ1.dot(X.T)
    db1 = 1 / m * np.sum(dZ1)
    return dw1, db1, dw2, db2

In [ ]:
def update_params(w1, b1, w2, b2, dw1, db1, dw2, db2, alpha):
    w1 = w1 - alpha * dw1
    b1 = b1 - alpha * db1
    w2 = w2 - alpha * dw2
    b2 = b2 - alpha * db2
    return w1, b1, w2, b2

In [ ]:
def get_prediction(A2):
    return np.argmax(A2, 0)

def get_accuracy(predictions, Y):
    print(predictions, Y)
    return np.sum(predictions == Y) / Y.size
    
def gradient_descent(X, Y, iters, alpha):
    w1, b1, w2, b2 = init_params()
    for i in range(iters):
        Z1, A1, Z2, A2 = forward_prop(w1, b1, w2, b2, X)
        dw1, db1, dw2, db2 = backward_prop(w2, Z1, A1, Z2, A2, X, Y)
        w1, b1, w2, b2 = update_params(w1, b1, w2, b2, dw1, db1, dw2, db2, alpha)
        if i % 50 == 0:
            print(f'Iteration: {i}')
            print(f'Accuracy: {get_accuracy(get_prediction(A2), Y)}')
    return w1, b1, w2, b2

In [ ]:
w1, b1, w2, b2 = gradient_descent(X_train, Y_train, 500, 0.1)